In [1]:
from transformers import logging
from transformers import BertModel, BertTokenizer
from transformers import pipeline
import torch
import torch.nn as nn
from torch.nn.functional import normalize, log_softmax

In [2]:
logging.set_verbosity_error()
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
from TRExData.LamaTRExData import LamaTRExData

from ModelHelpers.fill_mask_helpers import get_probability_from_pipeline_for_token
from relation_templates.templates import get_templates, get_relation_meta, relations, relation_names, get_relations_with_last_digit
from PipelineCacheWrapper.PipelineCacheWrapper import PipelineCacheWrapper

In [4]:
from SentenceComparison.SentenceComparison import SentenceComparison 
from ResultData.ResultPersister import ResultPersister

In [5]:
model='bert-base-uncased'
data_source = "complete"
TOP_K = 500

In [6]:
USELESS_WORD = "Erftwerk"
UNKNOWN_TOKEN = "[UNK]"
MASK = "[MASK]"
OBJ_LABEL = "obj-label"
SUBJ_LABEL = "sub-label"
VOCABULARY_SIZE = 28996
words = [] 

In [7]:
#relations = ["P140", "P127", "P36", "P159"]
#relations = ["P19"]
#relations = ["P19", "P413", "P159", "P103"]
relations = get_relations_with_last_digit(7)
len(relations)

8

In [8]:
%%time
unmasker = pipeline('fill-mask', model="bert-base-cased", top_k=VOCABULARY_SIZE)
#unmasker = PipelineCacheWrapper('fill-mask', model=model, top_k=TOP_K)

CPU times: user 1.1 s, sys: 209 ms, total: 1.31 s
Wall time: 10.6 s


In [9]:
TREx = LamaTRExData(relations = relations)
TREx.load()

In [10]:
@torch.no_grad()
def metric(sentence: str, token: str):
    prob = get_probability_from_pipeline_for_token(unmasker(sentence), token)
    return prob

In [11]:
Comparer = SentenceComparison(relations, get_templates, metric, MASK, get_relation_meta)

In [12]:
Comparer.compare(TREx.data)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 976/976 [28:10<00:00,  1.73s/it]


In [13]:
%%time
#unmasker.save_to_cache()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 2.15 µs


In [14]:
measure = "correct token probability"
persister = ResultPersister(model=model, measure=measure, data_source=data_source)

In [15]:
persister.persist(Comparer.results_for_persistence())
persister.close()

/Users/tim/projects/ContextualKnowledgeOfLMs/ma-env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/tim/projects/ContextualKnowledgeOfLMs/ma-env/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
